In [ ]:
import pyptvgtfs
import geopandas as gpd 
import pandas as pd
import os
from shapely.geometry import Point
from tqdm import tqdm

In [ ]:
ptv = pyptvgtfs.process_gtfs_zip('http://data.ptv.vic.gov.au/downloads/gtfs.zip', '')
ptv = ptv.set_index(['branch_id', 'table_name'], inplace=False)['df']

In [ ]:
gdf : gpd.GeoDataFrame = gpd.GeoDataFrame(pd.concat([gpd.read_file(f'../data/core/mb-geojson/{file}') for file in os.listdir('../data/core/mb-geojson')], ignore_index=True))


In [ ]:
branch_ids = ptv.index.get_level_values('branch_id').unique()

In [ ]:
for branch_id in branch_ids:
    ptv[branch_id]['stops']['branch_id'] = branch_id

In [ ]:
[ptv[branch_id]['stops']['stop_id'].is_unique for branch_id in branch_ids]

In [ ]:
df_stops : pd.DataFrame = pd.concat([ptv[branch_id]['stops'] for branch_id in branch_ids])

In [ ]:
df_stops[df_stops['stop_id'].duplicated(keep=False)].sort_values(by='stop_id')

In [ ]:
point0 = df_stops.iloc[0]
point0 = Point(point0['stop_lon'], point0['stop_lat'])

In [ ]:
gdf[gdf.geometry.contains(point0)]['MB_CODE21'].unique()

In [ ]:
df_stops_mb = df_stops.apply(lambda row: gdf[gdf.geometry.contains(Point(row['stop_lon'], row['stop_lat']))]['MB_CODE21'].unique(), axis=1)

In [ ]:
df_stops_points = df_stops.set_index(['stop_id', 'branch_id']).apply(axis=1, func=lambda row: Point(row['stop_lon'], row['stop_lat']))

In [ ]:
def temp(p):
    print(p)
    return gdf[gdf.geometry.contains(p)]['MB_CODE21'].unique()
df_stmb = df_stops_points.apply(temp)